<h2>Setup & Import</h2>

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# Import data and setup dataframe
df = pd.read_csv('data/T_T100_SEGMENT_ALL_CARRIER.csv')

In [12]:
# Import lookup tables
carrier_classes = pd.read_csv('data/L_CARRIER_GROUP_NEW.csv')
area_codes_table = pd.read_csv('data/L_WORLD_AREA_CODES.csv')
aircraft_types = pd.read_csv('data/L_AIRCRAFT_TYPE.csv')

city_markets = pd.read_csv('data/L_CITY_MARKET_ID.csv')
service_classes = pd.read_csv('data/L_SERVICE_CLASS.csv')
aircraft_groups = pd.read_csv('data/L_AIRCRAFT_GROUP.csv')
aircraft_congifs = pd.read_csv('data/L_AIRCRAFT_CONFIG.csv')

<h2>Data Cleansing</h2>

<h4>Carrier Classes</h4>

In [9]:
# Create simplified list of carrier class descriptions
carrier_classes_modified = []
for carrier_type in carrier_classes['Description']:
    index = carrier_type.index('Carrier')
    substr = carrier_type[:index-1]
    carrier_classes_modified.append(substr)

carrier_classes['Carrier Class'] = carrier_classes_modified

In [11]:
#Inspect updated carrier_classes
carrier_classes

,Code,Description,Carrier Class
0,0,Foreign Carriers,Foreign
1,1,Large Regional Carriers (carriers with annual ...,Large Regional
2,2,National Carriers (carriers with annual revenu...,National
3,3,Major Carriers (carriers with annual revenue o...,Major
4,4,Medium Regional Carriers (carriers with annual...,Medium Regional
5,5,Small Certificated Carriers (carrier holding c...,Small Certificated
6,6,Commuter Carriers (air taxi operator which per...,Commuter
7,7,All Cargo Carriers operating under cerificates...,All Cargo
8,9,Commuter Carriers (Air Taxi providing Essentia...,Commuter


In [7]:
# Replace carrier groups with descriptions from modified lookup table
df = df.merge(carrier_classes, left_on='CARRIER_GROUP_NEW', right_on='Code', how='left')
df = df.drop(columns=['Code', 'Description', 'CARRIER_GROUP_NEW'])

<h4>World Area Codes (WAC)</h4>

In [ ]:
# Replace WAC with descriptions from lookup table
df = df.merge(area_codes_table, left_on='ORIGIN_WAC', right_on='Code', how='left')
df['ORIGIN_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

df = df.merge(area_codes_table, left_on='DEST_WAC', right_on='Code', how='left')
df['DEST_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

<h4>Aircraft type</h4>

In [20]:
aircraft_types

,Code,Description
0,7,Aero Commander 200
1,8,Aero Macchi AL-60
2,9,Aeronca 7-AC
3,10,Beech Bonanza 35A/C/D/E/G/H/J/K/S/V/ 36A
4,20,Bellanca CH-300
...,...,...
439,887,B787-800 Dreamliner
440,888,Boeing 737-900ER
441,889,B787-900 Dreamliner
442,890,Antonov 225 (6 Engine)


In [22]:
df=['Description'] = df=['Description'].astype('string')

SyntaxError: can't assign to literal (227914532.py, line 1)

In [ ]:
df = df.merge(area_codes_table, left_on='AIRCRAFT_TYPE', right_on='Description', how='left')
#df['ORIGIN_WAC'] = df['Description']
#df = df.drop(columns=['Code', 'Description'])
df[:10]

<h2>Add Load Factor Metric</h2>

In [ ]:
#Add Load Factor column on flight level
df['Load Factor'] = df['PASSENGERS'] / df['SEATS']
df['Load Factor'] = df['Load Factor'].fillna(0)

<h2>Display Final Table</h2>

In [17]:
df[:10]

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Carrier Class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,30613,BID,"Block Island, RI",Rhode Island,United States,15,35855,WST,"Westerly, RI",Rhode Island,United States,15,0,79,1,2024,1,1,1,F,DU,Commuter
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,30613,BID,"Block Island, RI",Rhode Island,United States,15,35855,WST,"Westerly, RI",Rhode Island,United States,15,0,79,1,2024,3,7,1,F,DU,Commuter
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,35855,WST,"Westerly, RI",Rhode Island,United States,15,30613,BID,"Block Island, RI",Rhode Island,United States,15,0,79,1,2024,1,1,1,F,DU,Commuter
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,35855,WST,"Westerly, RI",Rhode Island,United States,15,30613,BID,"Block Island, RI",Rhode Island,United States,15,0,79,1,2024,3,7,1,F,DU,Commuter
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,13,32250,HYA,"Hyannis, MA",Massachusetts,United States,13,1,125,1,2024,1,1,1,F,DU,Commuter
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,13,30154,ACK,"Nantucket, MA",Massachusetts,United States,13,1,125,1,2024,1,2,1,F,DU,Commuter
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,31703,HPN,"White Plains, NY",New York,United States,22,31703,JFK,"New York, NY",New York,United States,22,6,622,1,2024,1,2,1,L,DU,Major
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,30721,BOS,"Boston, MA",Massachusetts,United States,13,30721,PVD,"Providence, RI",Rhode Island,United States,15,6,622,1,2024,3,7,1,L,DU,Major
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,34992,STX,"Christiansted, VI",U.S. Virgin Islands,United States,4,35626,VQS,"Vieques, PR",Puerto Rico,United States,3,1,125,1,2024,2,4,1,F,DU,Commuter
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,33342,MKE,"Milwaukee, WI",Wisconsin,United States,45,30977,MDW,"Chicago, IL",Illinois,United States,41,6,622,1,2024,2,4,1,L,DU,Major
